In [1]:
# 문자열 출력 파서 불러오기
from langchain_core.output_parsers import StrOutputParser

# 질의 확장을 적용할 사용자의 질문
query_for_nyc = "뉴욕은?"

# query augmentation
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages",),
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자가 질문한 의도를 파악해서 한 문장의 명료한 질문으로 변환하라. 대명사나 이, 저, 그와 같은 표현을 명확한 명사로 표현하라. :\n\n{query}"
        )
    ]
)

In [4]:
# 확장 쿼리 프롬프트와 챗봇과 StrOutputParser 로 체인 형성
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser() # StrOutputParser 인스턴스를 생성해서 넣어야 한다.

In [12]:
# %pip install nbformat 

# query_augmentation_chain 사용을 위해 사용자의 질문과 이전 대화 내용을 함께 제공해야 한다.
%run rag_practice_usedb.ipynb
# 이제 이 파일에서 정의된 chat_history 사용 가능


Loading existing Chroma store
서울시는 온실가스 저감을 위해 다양한 정책을 추진하고 있습니다. 주요 내용은 다음과 같습니다:

1. **정량적 감축 목표**: 서울시는 2050년 탄소중립을 목표로 하며, 2030년까지 2005년 대비 40%, 2040년까지 70%의 온실가스를 감축할 계획입니다.

2. **건물 부문 감축**: 서울의 온실가스 배출량의 90%를 차지하는 건물과 수송 부문에서 적극적인 대책을 마련하고 있습니다. 이를 위해 2026년까지 100만 호의 건물 에너지효율화 사업을 추진하고, 제로에너지 건물(ZEB) 의무화를 통해 에너지 절약을 촉진합니다.

3. **수송 부문 감소 조치**: 전기차 비중을 2026년까지 10%로 확대하고, 22만기의 전기차 충전 인프라를 구축할 예정입니다.

4. **기후예산제 도입**: 서울시는 모든 예산 사업에 온실가스 감축 방안을 고려하여 2021년부터 기후예산제 시범사업을 진행 중입니다.

5. **시민 참여와 협력적 거버넌스 구축**: 시민, 기업 등 다양한 도시 내 행위자와의 협력을 통해 탄소중립 목표를 달성할 수 있는 포용적인 거버넌스 체계를 구축하고 있습니다.

6. **녹색 인프라 확충**: 도심 속 녹색공간 조성 및 자원 순환 기반을 마련해 온실가스와 미세먼지 배출량을 저감하려고 하고 있습니다.

이와 같은 정책을 통해 서울시는 지속 가능한 도시로 거듭나기 위해 노력하고 있습니다.
content='서울시의 온실가스 저감 정책에 대해 알려줘.' additional_kwargs={} response_metadata={}
content='서울시는 온실가스 저감을 위해 다양한 정책을 추진하고 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **정량적 감축 목표**: 서울시는 2050년 탄소중립을 목표로 하며, 2030년까지 2005년 대비 40%, 2040년까지 70%의 온실가스를 감축할 계획입니다.\n\n2. **건물 부문 감축**: 서울의 온실가스 배출량의 90%를 차지하는 건

In [ ]:
# 질문을 더 명확하게
# 확장 쿼리(query augmentation)는 chat history 기반으로 GPT가 질문을 더 명확하게 재작성해주는 것이에요.
augmented_query = query_augmentation_chain.invoke({
    "messages": chat_history.messages,
    "query": query_for_nyc,
})

print(augmented_query)

뉴욕시의 온실가스 저감 정책에 대해 알려주세요.


In [14]:
from langchain_chroma import Chroma
import os

from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

embedding = OpenAIEmbeddings(model='text-embedding-3-large', api_key=OPEN_API_KEY)

persist_directory = '../chroma_store'

vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
    )

# 리트리버 생성
retriever = vectorstore.as_retriever(k=3)

In [15]:
# 확장 커리로 벡터 db에서 docs 가져오기
docs = retriever.invoke(augmented_query)

for d in docs:
    print(d)
    print('-----')

page_content='New York City, because of its density and public transportation 
system, has long had a smaller per capita carbon footprint than 
any other big city in the United States — and we have made 
significant progress reducing GHG emissions over the last decade, 
using new technologies and innovations to get us there. The City 
has assumed a leading global role in fighting climate change, 
and the actions we take can become a national and global model. 
However, the lack of commitment by the federal government 
to the Paris Agreement has placed New York and the world in a 
precarious position. Time is running out.
While New York City has made strides to achieve a reduction in 
greenhouse gas emissions, global emissions continue to rise, 
putting New Yorkers at risk. 
CHANGE IN GHG EMISSIONS, 2005-2017
Source: Mayor’s Office, International Energy Agency
0%
5%
GLOBAL
NEW YORK CITY-5%
15%
-15%
-20%
2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
10%
-10%
20%
25%' m

In [ ]:
# 언어 모델에서 답변 생성하기
chat_history.add_user_message(query_for_nyc) # 사용자의 질문 원본 "뉴욕은?" 추가

# 똑같이 뉴욕은? 이라고 질문해도 docs에 찾아둔 청크를 참조하기 때문에 제대로 된 대답 가능
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs
    }
)

chat_history.add_ai_message(answer)

print(answer)

뉴욕시는 온실가스 저감을 위한 여러 가지 정책과 프로그램을 시행하고 있습니다. 주요 내용은 다음과 같습니다:

1. **온실가스 감축 목표**: 뉴욕시는 2050년까지 탄소 중립을 달성하고, 2030년까지 2005년 대비 40%의 온실가스를 감축하겠다는 목표를 설정했습니다.

2. **예산 투자 및 에너지 효율성**: 뉴욕시는 에너지 효율성 개선을 위해 수십억 달러를 투자하고 있으며, 1,600개 이상의 공공건물에서 에너지 절약 조치를 시행하여 연간 6천만 달러 이상의 비용 절감 효과를 보고 있습니다.

3. **청정 에너지 및 건물 규정**: 2017년에는 새로운 건물이 제로 에너지 기준에 맞춰 건설되도록 요구하는 법안을 도입했습니다. 또한, 모든 신축 건물에서 최신 에너지 효율성을 적용하도록 하고 있습니다.

4. **커뮤니티 리트로핏 프로그램**: 뉴욕시는 여러 프로그램을 통해 중소형 건물, 병원 및 다세대 주택 등이 온실가스를 줄이도록 지원하고 있습니다.

5. **GreeNYC 캠페인**: 개인과 기업이 탄소 발자국을 줄이도록 장려하는 GreeNYC 캠페인을 통해, 시민들이 환경을 보호하고 친환경적인 삶을 살도록 유도하고 있습니다.

6. **시민 참여**: 환경정의 지침을 모든 프로젝트에 적용하고, 시민의 참여를 촉진하여 기후 변화에 대한 대응 방안을 마련하고 있습니다.

이러한 노력들을 통해 뉴욕시는 지속 가능한 도시로 나아가고 있으며, 전 세계의 기후 변화 대응 모델이 되고자 하고 있습니다.
